In [1]:
# Let's import all required libraries

import selenium                                  #library that is used to work with selenium

from selenium import webdriver                   #importing webdriver module from selenium to open automated chrome window

import pandas as pd                              #to create DataFrame

from selenium.webdriver.common.by import By      #importing inbuilt class By 

import warnings                                  #to ignore any sort of warning

warnings.filterwarnings("ignore")

import re

import time                                      #use to stop search engine for few seconds

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service

from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import requests
from selenium.common.exceptions import NoSuchElementException

#### 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [228]:

driver = webdriver.Edge(EdgeChromiumDriverManager().install())
driver.get('https://www.amazon.in/')
driver.maximize_window()

#take input from user
search_item=input("Enter the product name to be searched :")
search=driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
search.send_keys(search_item)

driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input').click()

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.200/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.200]


Enter the product name to be searched :guitar


# 

#### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [231]:
#fetching 3 pages url 
producturl=[]

for i in range(0,3):# to fetch product details from 3 pages
    urls=driver.find_elements_by_xpath('//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for url in urls:
        producturl.append(url.get_attribute('href'))
    next=driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')  
    next.click()
    time.sleep(2)

In [232]:
len(producturl)

148

In [233]:
product_details={'product_name':[], 'Price':[], 'Return_Exchange':[],'Expected_Delivery':[],
                 'Availability':[],'Product_URL':[]}
                 
for product in producturl:
    driver.get(product)
    time.sleep(2)
    
    #product name
    try:                    
        name=driver.find_element_by_xpath('//div[@id="titleSection"]/h1/span').text
        product_details['product_name'].append(name)
    except NoSuchElementException :
         product_details['product_name'].append('-')
    
    #price
    try:
        #price=driver.find_element_by_xpath('//span[@class="a-price-whole"]').text
        price=driver.find_element_by_xpath('//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]/span[2]').text
        product_details['Price'].append(price)
    except NoSuchElementException :
        
         product_details['Price'].append('-')
#Return_Exchange
    try:
        ReturnExchange=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[5]/div[3]/div[4]/div[24]/div[2]/div/div/div/div[2]/div/ol/li[3]/div/span/div[2]/span').text
        product_details['Return_Exchange'].append(ReturnExchange)
    except NoSuchElementException :
         product_details['Return_Exchange'].append('-')

#Expected_Delivery
    try:
        ExpectedDelivery=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[5]/div[3]/div[1]/div[3]/div/div[1]/div/div/div/form/div/div/div/div/div[3]/div/div[2]/div[9]/div[1]/div/div/div[2]/span/span[1]').text
        product_details['Expected_Delivery'].append(ExpectedDelivery)
    except NoSuchElementException :
         product_details['Expected_Delivery'].append('-')
#Availability
    try:
        available=driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[5]/div[3]/div[1]/div[3]/div/div[1]/div/div/div/form/div/div/div/div/div[3]/div/div[4]/div/div[1]/span').text
        #available=driver.find_element_by_xpath('//div[@id="availability"]')
        product_details['Availability'].append(available)
    except NoSuchElementException :
         product_details['Availability'].append('-')
            
#Product_URL
    product_details['Product_URL'].append(product)
    

In [235]:
df=pd.DataFrame(product_details)
df

,product_name,Price,Return_Exchange,Expected_Delivery,Availability,Product_URL
0,Imaginea® 40 inch Acoustic Guitar for Beginner...,"₹5,449",7 days Returnable,-,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Ibanez MD39C-NT 39 Inches Rose Wood Cutaway Ac...,"₹7,699",-,-,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,INTERN 41 inches Cutaway Shape Acoustic Guitar...,"₹3,290",-,-,-,https://www.amazon.in/Acoustic-plectrums-stabi...
3,Yamaha FX280 Tobacco Brown Sunburst Electro Ac...,"₹10,999",-,-,-,https://www.amazon.in/FX280-acoustic-Dreadnoug...
4,Fender Squier 39 Inch 6 String Cutaway Acousti...,"₹7,400",-,-,-,https://www.amazon.in/Fender-String-Cutaway-Ac...
...,...,...,...,...,...,...
143,Vault Junior 30 Inch Acoustic Guitar for Kids ...,"₹3,999",7 days Replacement,"Tomorrow, 12 August",In stock,https://www.amazon.in/Junior-Acoustic-Guitar-S...
144,"Epiphone, Acoustic-Electric Guitar, AJ-220SCE ...","₹34,009",-,-,-,https://www.amazon.in/Epiphone-Acoustic-Electr...
145,"YAMAHA Pacifica012 Electric Guitar, Red Metallic","₹14,762",-,-,-,https://www.amazon.in/Yamaha-PAC012-Electric-G...
146,Epiphone SG Muse 6 String Right Handed Electri...,"₹44,484",-,-,-,https://www.amazon.in/Epiphone-Electric-Inspir...


In [ ]:
Availability=driver.find_element_by_xpath('//div[@id="availability"]').text
print(Availability)
    

In [ ]:
#closing the window
driver.close()

#### 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [53]:
def img_download(url,search):
    
    driver = webdriver.Edge(EdgeChromiumDriverManager().install())
    driver.get(url)
    driver.maximize_window()
    search=driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea')
    search.send_keys(search)
        #click search                 
    driver.find_element_by_xpath(' /html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input').click()
    time.sleep(1)

    for _ in range(20):
            driver.execute_script('window.scrollBy(0,100)')

            images=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

            img_urls=[]
            img_data=[]
            for image in images:
                source=image.get_attribute('src')
                if source is not None:
                    if(source[0:4]=='http'):
                        img_urls.append(source)

            for i in range(0,10):
                print(f"Downloading {listitem}{i} of 10 images")
                response=requests.get(img_urls[i])
                file=open(r"D:\vaishali\Datatrained\datatrained23-24\Intrenship fliprobo\assignment7_ExceptionHandling_webscraping\Images_Ques3"+str(i)+".jpg","wb")
                file.write(response.content)  
    time.sleep(1)     

In [69]:
driver = webdriver.Edge(EdgeChromiumDriverManager().install())
url='https://images.google.com/'
driver.get(url)
driver.maximize_window()

#list of items
search_list=['fruits','cars','Machine','Learning' ,'Guitar', 'Cakes']
k=0
for item in search_list:
    driver.get(url)
    time.sleep(2)
   
    search_bar = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea")
    search_bar.send_keys(str(item))
        # clicking on search button
    search_button = driver.find_element_by_xpath("//button[@class='Tg7LZd']").click()
    time.sleep(1)
    for _ in range(20):
        driver.execute_script('window.scrollBy(0,100)')

        images=driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

        img_urls=[]
        img_data=[]
        for image in images:
            source=image.get_attribute('src')
            if source is not None:
                if(source[0:4]=='http'):
                    img_urls.append(source)

    for i in range(0,10):
        k=k+1 #it should not replace previously downloaded item coz having same path
        print(f"Downloading {item}{i} of 10 images")
        response=requests.get(img_urls[i])
        file=open(r"D:\vaishali\Datatrained\datatrained23-24\Intrenship fliprobo\assignment7_ExceptionHandling_webscraping\Images_Ques3\"+str(i)+str(k)+".jpg","wb")
        file.write(response.content)  
    time.sleep(1)     

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.200/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.200]


In [ ]:
#closing the window
driver.close()

# 

#### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped:
“Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [90]:

driver = webdriver.Edge(EdgeChromiumDriverManager().install())

driver.get('https://www.flipkart.com/')
driver.maximize_window()

#login Pop up window
try:
    driver.find_element_by_xpath("/html/body/div[2]/div/div/button").click()
except NoSuchElementException:
    print("")

time.sleep(1)

#search bar

search=driver.find_element_by_tag_name('input')
search.send_keys('smartphone')
#click search button
driver.find_element_by_xpath('//button[@class="L0Z3Pu"]').click()

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.200/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.200]


In [94]:

item_urls=[]
urls=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    item_urls.append(url.get_attribute('href'))
    

In [95]:
len(item_urls)

24

In [96]:
#initializing Dictionary to store data
phone_Details={'Brand_Name':[], 'Smartphone_name':[], 'Colour':[], 
               'RAM':[], 'Storage_ROM':[], 'Primary_Camera':[],
               'Secondary_Camera':[], 'Display_Size':[], 'Battery_Capacity':[],
               'Price':[], 'Product_URL':[]}
for url in item_urls:
    driver.get(url)
    time.sleep(2)
    #click read more button             
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    time.sleep(0.5)
    #Brand_Name
    try:                                  
        name=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[3]/div/div[1]/h1/span')
        
        phone_Details['Brand_Name'].append(name.text.split(' ')[0])
    except NoSuchElementException :
         phone_Details['Brand_Name'].append('-')
   
    #Smartphone_name        
    try:                                        
        phonename=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[3]/td[2]/ul/li').text
       
        phone_Details['Smartphone_name'].append(phonename)
    except NoSuchElementException :
         phone_Details['Smartphone_name'].append('-')
    
    #Colour        
    try:                    
        color=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[1]/table/tbody/tr[4]/td[2]/ul/li').text
        
        phone_Details['Colour'].append(color)
    except NoSuchElementException :
         phone_Details['Colour'].append('-')
    
    # RAM
    try:                    
        ram=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[2]/td[2]/ul/li').text
        phone_Details['RAM'].append(ram)
        
    except NoSuchElementException :
         phone_Details['RAM'].append('-')    
    
    # Storage_ROM
    try:                    
        rom=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[4]/table/tbody/tr[1]/td[2]/ul/li').text
        phone_Details['Storage_ROM'].append(rom)
        
    except NoSuchElementException :
         phone_Details['Storage_ROM'].append('-')
    
    # Primary_Camera
    try:                    
        prCam=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[2]/td[2]/ul/li').text
        phone_Details['Primary_Camera'].append(prCam)
        
    except NoSuchElementException :
         phone_Details['Primary_Camera'].append('-')
            
    # Secondary_Camera
    try:                    
        srCam=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[5]/table/tbody/tr[6]/td[2]/ul/li').text
        phone_Details['Secondary_Camera'].append(srCam)
        
    except NoSuchElementException :
         phone_Details['Secondary_Camera'].append('-')
            
    # Display_Size
    try:                    
        dispaly=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[2]/table/tbody/tr[1]/td[2]/ul/li').text
        phone_Details['Display_Size'].append(dispaly)
        
    except NoSuchElementException :
         phone_Details['Display_Size'].append('-')
            
    # Battery_Capacity
    try:                    
        batCap=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[10]/div[5]/div/div[2]/div[1]/div[10]/table/tbody/tr[1]/td[2]/ul/li').text
        phone_Details['Battery_Capacity'].append(batCap)
        
    except NoSuchElementException :
         phone_Details['Battery_Capacity'].append('-')
            
    # Price
    try:                    
        price=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div[2]/div[3]/div/div[4]/div[1]/div/div[1]').text
        phone_Details['Price'].append(price)
       
    except NoSuchElementException :
         phone_Details['Price'].append('-')
    phone_Details['Product_URL'].append(url)

In [97]:
df=pd.DataFrame(phone_Details)
df

,Brand_Name,Smartphone_name,Colour,RAM,Storage_ROM,Primary_Camera,Secondary_Camera,Display_Size,Battery_Capacity,Price,Product_URL
0,SAMSUNG,Galaxy F13,Waterfall Blue,4 GB,64 GB,50MP + 5MP + 2MP,8MP Front Camera,16.76 cm (6.6 inch),6000 mAh,"₹9,599",https://www.flipkart.com/samsung-galaxy-f13-wa...
1,SAMSUNG,Galaxy F04,Jade Purple,4 GB,64 GB,13MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),5000 mAh,"₹6,999",https://www.flipkart.com/samsung-galaxy-f04-ja...
2,SAMSUNG,Galaxy F13,Sunrise Copper,4 GB,64 GB,50MP + 5MP + 2MP,8MP Front Camera,16.76 cm (6.6 inch),6000 mAh,"₹9,599",https://www.flipkart.com/samsung-galaxy-f13-su...
3,SAMSUNG,Galaxy F13,Waterfall Blue,4 GB,128 GB,50MP + 5MP + 2MP,8MP Front Camera,16.76 cm (6.6 inch),6000 mAh,"₹10,599",https://www.flipkart.com/samsung-galaxy-f13-wa...
4,POCO,C51,Power Black,4 GB,64 GB,8MP Dual Rear Camera,5MP Front Camera,16.56 cm (6.52 inch),5000 mAh,"₹6,499",https://www.flipkart.com/poco-c51-power-black-...
5,POCO,C55,Forest Green,6 GB,128 GB,50MP Dual Rear Camera,5MP Front Camera: Camera Features: Portrait Mo...,17.04 cm (6.71 inch),5000 mAh,"₹8,499",https://www.flipkart.com/poco-c55-forest-green...
6,POCO,C51,Royal Blue,4 GB,64 GB,8MP Dual Rear Camera,5MP Front Camera,16.56 cm (6.52 inch),5000 mAh,"₹6,499",https://www.flipkart.com/poco-c51-royal-blue-6...
7,vivo,T2x 5G,Aurora Gold,4 GB,128 GB,50MP + 2MP,8MP Front Camera Setup: (f/2.0 Aperture),16.71 cm (6.58 inch),5000 mAh,"₹12,999",https://www.flipkart.com/vivo-t2x-5g-aurora-go...
8,POCO,C55,Power Black,6 GB,128 GB,50MP Dual Rear Camera,5MP Front Camera: Camera Features: Portrait Mo...,17.04 cm (6.71 inch),5000 mAh,"₹8,499",https://www.flipkart.com/poco-c55-power-black-...
9,vivo,T2x 5G,Marine Blue,6 GB,128 GB,50MP + 2MP,8MP Front Camera Setup: (f/2.0 Aperture),16.71 cm (6.58 inch),5000 mAh,"₹13,999",https://www.flipkart.com/vivo-t2x-5g-marine-bl...


In [98]:
# saving the data in csv
df.to_csv("smartphones.csv")

In [99]:
#closing the window
driver.close()

# 

#### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [104]:
# connecting to the webdriver
driver = webdriver.Edge(EdgeChromiumDriverManager().install())
#opening the url
driver.get('https://www.google.co.in/maps')

#maximize the window
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.200/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.200]


In [106]:
#taking input from user
city_name=input("Enter city :")
#passing it to search tab
search_city=driver.find_element_by_tag_name("input")

#passing user input to tab
search_city.send_keys(city_name)
time.sleep(1)

#click search button
driver.find_element_by_xpath('//div[@class="pzfvzf"]/button').click()
time.sleep(1)

#get current url
cur_url=driver.current_url
print(cur_url)

Enter city :pune
https://www.google.co.in/maps/place/Pune,+Maharashtra/@18.524545,73.6981538,11z/data=!4m6!3m5!1s0x3bc2bf2e67461101:0x828d43bf9d9ee343!8m2!3d18.5204303!4d73.8567437!16zL20vMDE1eTJx?entry=ttu


In [111]:
#get current url
urlsplit=cur_url.split('@')
data=urlsplit[1].split(',')[:2]
latitude=data[0]
longitude=data[1]

print(f"Latitude of {city_name} is: {latitude}  and Latitude is :{longitude}")

Latitude of pune is: 18.524545  and Latitude is :73.6981538


In [112]:
#close the window
driver.close()

#### 6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [120]:
url= 'https://www.digit.in./ '
driver = webdriver.Edge(EdgeChromiumDriverManager().install())
driver.get(url)
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.200/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.200]


In [121]:
 #click best gaming laptop link
gaming_link=driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[2]/div[4]/ul/li[9]/a').get_attribute('href')
driver.get(gaming_link)

In [145]:
#defining dict.
Laptops={'Name':[],'Processor':[],'Display':[],'OS':[],'Memory':[],
         'Graphics_Processor':[],'Body':[],'Price':[]}
#scraping names
try:
    names= driver.find_elements_by_xpath('//span[@class="datahreflink"]')
    
    for name in names[:7]:
        Laptops['Name'].append(name.text)
except :
    pass

#scraping Processor
try:
    processors= driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[2]/td[3]')
    
    for i in processors:
        Laptops['Processor'].append(i.text)
except :
    pass

#scraping Displays
try:
    Displays= driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
    
    for dis in Displays:
        Laptops['Display'].append(dis.text)
except :
    pass

#scraping OS
try:
    os= driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
    
    for j in os:
        Laptops['OS'].append(j.text)
except :
    pass

#scraping Memory
try:
    memories= driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
    
    for mem in memories:
        Laptops['Memory'].append(mem.text)
except :
    pass

#scraping Graphics_Processor
try:
    grp_pros= driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')
    
    for grp in grp_pros:
        Laptops['Graphics_Processor'].append(grp.text)
except :
    pass


#scraping Body
try:
    bodies= driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    
    for body in bodies:
        Laptops['Body'].append(body.text)
except :
    pass


#scraping Price
try:
    prices= driver.find_elements_by_xpath('//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
    
    for pr in prices:
        Laptops['Price'].append(pr.text)
except :
    pass

In [146]:
#making dataframe
df=pd.DataFrame(Laptops)
df

,Name,Processor,Display,OS,Memory,Graphics_Processor,Body,Price
0,HP OMEN 17 (2023),13th Gen Intel Core i7-13700HX 16 core process...,"17.3″ (2560 x 1440) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR5GB RAM & 1 TB SSD,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,397.1 x 262 x 27 mm dimension & 2.78 kg weight,"₹ 269,777"
1,MSI Titan GT77 12UHS,12th Gen Intel Core i9-12900HX 16 core process...,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",Windows 11 Home,64 GB DDR5GB RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...,397 x 330 x 23 mm dimension & 3.3 kg weight,"₹ 499,990"
2,Lenovo Legion 5i Pro,12th Gen Intel Core i7-12700H 14 core processo...,16″ (2560 x 1600) screen,Windows 11 Home,32 GB DDR5GB RAM & 1 TB SSD,NVIDIA GeForce RTX 3070 Ti Graphics card,359.9 x 264.4 x 19.9 mm dimension & 3.6 kg weight,"₹ 179,990"
3,ASUS ROG Strix Scar 18 2023,13th Gen Intel Core i9-13980HX 24 core process...,"18″ (1920 x 1200) screen, 165 Hz refresh rate",Windows 11 Home,32 GB DDR5GB RAM & 1 TB SSD,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,294 x 399 x 23 mm dimension & 3.1 kg weight,"₹ 279,990"
4,Acer Predator Helios Neo 16,13th Gen Intel Core i7-13700HX processor,"16″ (2560 x 1600) screen, 165 Hz refresh rate",Windows 11 Home,16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,360 x 279 x 28 mm dimension & 2.6 kg weight,"₹ 149,990"
5,ASUS ROG Zephyrus G14,AMD Ryzen 9-6900HS 8 core processor with 4.9 G...,"14″ (1920 x 1200) screen, 144 Hz refresh rate",Windows 11 Home,16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 AMD Radeon RX 6700S Graphics card,312 x 227 x 19 mm dimension & 1.65 kg weight,"₹ 156,990"
6,MSI Cyborg 15,12th Gen Intel Core i7-12650H 10 core processo...,"15.6″ (1920 x 1080) screen, 144 Hz refresh rate",Windows 11 Home,16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,& 1.98 kg weight,"₹ 125,000"


In [147]:
#close th window
driver.close()

# 

#### 7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [148]:
url= 'https://www.forbes.com./ '
driver = webdriver.Edge(EdgeChromiumDriverManager().install())
driver.get(url)
driver.maximize_window()

time.sleep(1)



[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.200/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.200]


In [149]:
#click drop down
try:
    driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/div[1]/div/div').click()

except NoSuchElementException as e:
    print("Exception raised :",e)
    driver.find_element_by_xpath('//div[@class="_69hVhdY4"]').click()
time.sleep(1)



In [150]:
#click bellaneries option 
try:
    driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]/div[1]').click()

except NoSuchElementException as e:
    print("Exception raised :",e)
    #driver.find_element_by_xpath('//div[@class="_69hVhdY4"]').click()
time.sleep(1)


In [ ]:
#click view all Billionaires    
try:
    driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/div[1]/div/div[2]/ul/li[2]/div[2]/div[2]/a').click()
except Exception as e:
    print("Exception raised :",e)
    driver.find_element_by_xpath('//ul[@class="E-S7Afho"]/li[2]').click()
    

In [151]:
# worlds billionaries list click
try:
    driver.find_element_by_xpath('/html/body/div[1]/div/main/div/section/div[2]/div/div/div[1]/div/div[1]/div/div/a').click()
except Exception as e:
    print("Exception raised :",e)
    driver.find_element_by_xpath('//ul[@class="E-S7Afho"]/li[2]').click()

In [152]:
Billionaires={'Rank':[], 'Name':[], 'Net_worth':[], 'Age':[], 'Citizenship':[], 'Source':[], 'Industry':[]}
#first row   ( //div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"])

data=driver.find_elements_by_xpath('//div[@class="TableRow_row__L-0Km TableRow_activeRow__g4oSF"]/div')
Billionaires['Rank'].append(data[0].text)
Billionaires['Name'].append(data[1].text)
Billionaires['Net_worth'].append(data[2].text)
Billionaires['Age'].append(data[3].text)
Billionaires['Citizenship'].append(data[4].text)
Billionaires['Source'].append(data[5].text)
Billionaires['Industry'].append(data[6].text)
    

In [153]:


rows=driver.find_elements_by_xpath('//div[@class="TableRow_rowContainer__IC1Tv"]/div')
for i in range(0,len(rows)-1):
    row=[]
    data=rows[i].text.split('\n')
    
    if len(data)==7:
        Billionaires['Rank'].append(data[0])
        Billionaires['Name'].append(data[1])
        Billionaires['Net_worth'].append(data[2])
        Billionaires['Age'].append(data[3])
        Billionaires['Citizenship'].append(data[4])
        Billionaires['Source'].append(data[5])
        Billionaires['Industry'].append(data[6])

# making Dataframe    
df=pd.DataFrame(Billionaires) 
df

,Rank,Name,Net_worth,Age,Citizenship,Source,Industry
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,Fashion & Retail
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,$114 B,59,United States,Amazon,Technology
3,4,Larry Ellison,$107 B,78,United States,Oracle,Technology
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
194,195,Rick Cohen & family,$9.6 B,70,United States,Warehouse automation,Technology
195,195,Jin Baofang,$9.6 B,70,China,Solar panels,Energy
196,195,Luo Liguo & family,$9.6 B,67,China,Chemicals,Manufacturing
197,195,Marijke Mars,$9.6 B,58,United States,"Candy, pet food",Food & Beverage


In [154]:
driver.close()

# 

#### 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [218]:
url= 'https://www.youtube.com/ '
#establish connection with webdriver
driver = webdriver.Edge(EdgeChromiumDriverManager().install())

#opening the given url page
driver.get(url)
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.200/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.200]


In [221]:
#search bar
search=driver.find_element_by_xpath('/html/body/ytd-app/div[1]/div/ytd-masthead/div[4]/div[2]/ytd-searchbox/form/div[1]/div[1]/input')

#sending searching video
search.send_keys('BTS - Dynamite')

#clicking search button
try:
    driver.find_element_by_xpath('//button[@id="search-icon-legacy"]').click()
    
except :
    driver.find_element_by_xpath('/html/body/ytd-app/div[1]/div/ytd-masthead/div[4]/div[2]/ytd-searchbox/button').click()
    time.sleep(1)

In [222]:
# scraping the video link
try:
    video_link =driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[2]/div[1]/div/div[1]/div/h3/a').get_attribute('href')
    driver.get(video_link)
except:
    pass

In [223]:
# 5000 times we scroll down by 500  to load more comments
for _ in range(5000):
    driver.execute_script("window.scrollBy(0,500)")

In [224]:
Comments={'comment':[], 'Comment_upvote':[], 'Time':[]}
#scraping comment
try:
    comments=driver.find_elements_by_id("comment-content")
    for i in comments[:500]:
        Comments['comment'].append(i.text)
except :
    pass

#scraping Comment_upvote
try:
    upvotes=driver.find_elements_by_id("toolbar")
    
    for i in upvotes[:500]:
        upvote=i.find_element_by_id("vote-count-middle").text
       
        
        if upvote==" ":
            Comments['Comment_upvote'].append('-')
        else:
            Comments['Comment_upvote'].append(upvote)
except :
    pass

#scraping time
try:
    times=driver.find_elements_by_id("header-author")
    
    for i in times[:500]:
        time=i.text.split('\n')[1]
        
        
        if time==" ":
            Comments['Time'].append('-')
        else:
            Comments['Time'].append(time)
except :
    pass

In [225]:

print(len(Comments['Time']),len(Comments['Comment_upvote']),len(Comments['comment']))

500 500 500


In [226]:
#making dataframe
df=pd.DataFrame(Comments)
df

,comment,Comment_upvote,Time
0,This is not only a song of BTS\n\nThis is BTS ...,1.1K,2 years ago
1,レベルが高いんだよなぁ。本当に曲もいいし、ダンスもかっこいいし、何回でも聴きたくなる。,173,8 months ago
2,They are the pure definition of cool and I can...,43,2 months ago
3,この曲は辛い気分の時も、本当に元気が貰えます、大好きです。,277,7 months ago
4,I love this choregraphy,1,10 days ago
...,...,...,...
495,BRO WHY ARE THEY SOOOO HANDSOME AND CUTE,,7 months ago
496,リズムかいい,,4 months ago
497,Bts: how many dynamite videos do u want?\nArmy...,1.2K,2 years ago
498,En 2023 quién sige escuchando está obra de arte?,5,4 months ago


In [212]:
#close window
driver.close()

# 

### remaining privates from price, dorms from price

#### 9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [110]:
url= 'https://www.hostelworld.com/ '
driver = webdriver.Edge(EdgeChromiumDriverManager().install())
driver.get(url)
driver.maximize_window()



[WDM] - ====== WebDriver manager ======
[WDM] - Current edge version is 115.0.1901
[WDM] - Get LATEST edgedriver version for 115.0.1901 Edge
[WDM] - Trying to download new driver from https://msedgedriver.azureedge.net/115.0.1901.203/edgedriver_win64.zip
[WDM] - Driver has been saved in cache [C:\Users\nbson\.wdm\drivers\edgedriver\win64\115.0.1901.203]


In [112]:
#search bar
search=driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/input')

#sending city name to serach bar
search.send_keys('London')
time.sleep(1)

In [114]:
#select 1st drop down item
driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[2]/div/ul/li[2]').click()
time.sleep(2)
#click search button
driver.find_element_by_xpath('//button[@class="btn-content large-button icon-only"]').click()

In [125]:
#scrap all div containing details
divs=driver.find_elements_by_xpath('//div[@class="property-card"]')
len(divs)

32

In [102]:
#initializing dictionary
Details={'Name':[],'Dist':[],'Ratings':[],'Total_reviews':[],'Overall_reviews':[],
         'Privates_price':[],'Dorms_price':[],'Facilities':[],'Property_description':[]}


In [ ]:
Details['Dist']=[]
#scraping all details
for i in divs:
    #scrap dist
    try:
        dist=i.find_element_by_class_name('distance-description').text
        Details['Dist'].append(dist)
        #print(dist)
    except:
        Details['Dist'].append("-")


In [ ]:
Details['Dorms_price']=[] 
Details['Private_ price']=[]

#first 2 hotels dorms price
hotel_prs=driver.find_elements_by_xpath('//div[@class="accommodation-price"]')
for i in range(0,2):
    Details['Dorms_price'].append(hotel_prs[i].text)
    Details['Private_ price'].append('')

#scraping prices
pvt=driver.find_elements_by_xpath('//div[@class="property-accommodation-prices"]/div[1]')
dorms=driver.find_elements_by_xpath('//div[@class="property-accommodation-prices"]/div[2]')
print(len(pvt),len(dorms))

#Scraping dorms n private rates
for i in range(len(pvt)):
    #scraping pvt price
    pvt_prices=pvt[i].text
    #print(pvt_prices)
    
    if pvt_prices=='No Privates Available':
        Details['Private_ price'].append('-') 
        
    else:
       #pvt_prices comes with % discount and original price with discounted price so finding the 
        #original price and discounted price by using regex function 
        
        pvt_price_split=re.findall(r"₹[0-9,]*",pvt_prices)
        
        #converting pvt_price_split from pattern['₹15,357'] -->₹15,357 But here type of pvt_price is List so we can't split directly.will access ele. by index n then split 
        pvt_price=pvt_price_split[-1:]
        pvPr=pvt_price[0].split('['')[1].split('']')[0]
        Details['Private_ price'].append(pvPr)
    
    #scraping dorms price
    
    dorms_prices=dorms[i].text
    
    if dorms_prices=='No Dorms Available':
        dorm_price='-'
        Details['Dorms_price'].append(dorm_price)
    else:
        #dorms_prices comes with % discount and original price with discounted price so finding the 
        #original price and discounted price by using regex function 
        
        dorms_price_split=re.findall(r"₹[0-9,]*",dorms_prices)
        
        
        dorm_price= dorms_price_split[-1:]
        
        #converting dorms_price_split from pattern['₹15,357'] -->₹15,357 .But here type of dorm_price is List so we can't split directly.will access ele. by index n then split 
        dorm_pr=dorm_price[0].split('['')[1].split('']')[0]
        Details['Dorms_price'].append(dorm_pr)
    
print(len(Details['Dorms_price']) ,'\n', len(Details['Private_ price']))
    

In [118]:
#scraping pages link
page_links=[]
links=driver.find_elements_by_xpath('//a[@class="nuxt-link"]')
len(links)
for link in links:
    page_links.append(link.get_attribute('href'))
len(page_links)

32

In [ ]:
Details={'Name':[],'Ratings':[],'Total_reviews':[],'Overall_reviews':[],
         'Facilities':[],'Property_description':[]}

# scraping data from pages
for page in page_links:
    driver.get(page)
    
    #scrap name
    try:
        name=driver.find_element_by_class_name("title-2").text
        #print(name)
        Details['Name'].append(name)
    except:
        Details['Name'].append(" ")
        
    #scrap Ratings
    try:
        rating=driver.find_element_by_class_name("score orange big").text
        #print(rating)
        Details['Ratings'].append(rating)
    except:
        Details['Ratings'].append(" ")
        
    #scrap Total_reviews
    try:
        totalreview=driver.find_element_by_class_name("reviews").text
        #print(totalreview)
        Details['Total_reviews'].append(totalreview)
    except:
        Details['Total_reviews'].append(" ")
        
    #scrap Overall_reviews
    try:
        overallreview=driver.find_element_by_class_name("keyword").text
        #print(overallreview)
        Details['Overall_reviews'].append(overallreview)
    except:
        Details['Overall_reviews'].append(" ")
   
    #scrap Facilities
    try:
        facility=driver.find_element_by_class_name("facilities").text
        #print(facility)
        Details['Facilities'].append(facility)
    except:
        Details['Facilities'].append(" ")
        
     #scrap Property_description
    try:
        propDiscrip=driver.find_element_by_xpath("/html/body/div[3]/div/div/div[2]/section/div[9]/div/div[2]/div[4]/div/div[2]").text
        #print(propDiscrip)
        Details['Property_description'].append(propDiscrip)
    except:
        Details['Property_description'].append(" ")

In [130]:


print(len(Details['Name']),len(Details['Dist']),len(Details['Ratings']),len(Details['Overall_reviews']),len(Details['Total_reviews']),
      len(Details['Facilities']),len(Details['Property_description']))

32 32 32 32 32 32 32


In [131]:
df=pd.DataFrame(Details)
df

,Name,Dist,Ratings,Total_reviews,Overall_reviews,Facilities,Property_description
0,St Christopher's Village,-,,12307 Total Reviews,Fabulous,Linen Included Free WiFi Free Internet Access,
1,Clink261,-,,349 Total Reviews,Fabulous,Linen Included Free City Maps Free WiFi,
2,Wombat's City Hostel London,- 3.6km from city centre,,15149 Total Reviews,Superb,Linen Included Free City Maps Free WiFi Free I...,A safe haven in the middle of the metropolis: ...
3,Onefam Notting Hill by Hostel One,- 5.5km from city centre,,2179 Total Reviews,Superb,Linen Included Free WiFi Free Internet Access,The perfect place for solo travelers to connec...
4,St Christopher's Village,- 1.8km from city centre,,12307 Total Reviews,Fabulous,Linen Included Free WiFi Free Internet Access,
5,Generator London,- 3km from city centre,,7638 Total Reviews,Very Good,Linen Included Free City Maps Free WiFi Free I...,
6,NX London Hostel,- 6.1km from city centre,,2026 Total Reviews,Fabulous,Free Breakfast Linen Included Towels Included ...,Welcome to NX London Hostel!\n\nPLEASE READ IN...
7,Urbany Hostel London,- 5.4km from city centre,,852 Total Reviews,Superb,Linen Included Free WiFi,"Welcome to Urbany Hostel London, our first int..."
8,Safestay London Elephant & Castle,- 1.7km from city centre,,5043 Total Reviews,Very Good,Linen Included Free WiFi,
9,Pickwick Hall,- 2.3km from city centre,,2694 Total Reviews,Fabulous,Free Breakfast Linen Included Free Internet Ac...,Pickwick Hall provides accommodation for touri...


In [132]:
driver.close()